In [1]:
import os
import re
from collections import defaultdict

def parse_cube_files(luts_folder):
    """
    Parse .cube files and extract LUT_3D_SIZE values.
    """
    size_to_files = defaultdict(list)
    
    luts_path = os.path.join(os.getcwd(), luts_folder)
    
    if not os.path.exists(luts_path):
        print(f"Error: {luts_path} does not exist")
        return
    
    # Get all .cube files
    cube_files = [f for f in os.listdir(luts_path) if f.endswith('.cube')]
    
    if not cube_files:
        print(f"No .cube files found in {luts_path}")
        return
    
    # Parse each file
    for filename in cube_files:
        filepath = os.path.join(luts_path, filename)
        try:
            with open(filepath, 'r') as f:
                for line in f:
                    # Look for LUT_3D_SIZE line
                    match = re.match(r'LUT_3D_SIZE\s+(\d+)', line.strip())
                    if match:
                        size = match.group(1)
                        size_to_files[size].append(filename)
                        break
        except Exception as e:
            print(f"Error reading {filename}: {e}")
    
    # Display results
    print(f"\n{'LUT_3D_SIZE':<15} {'Count':<10} {'Files'}")
    print("=" * 60)
    
    for size in sorted(size_to_files.keys(), key=int):
        files = size_to_files[size]
        print(f"{size:<15} {len(files):<10} {', '.join(files[:3])}")
        if len(files) > 3:
            print(f"{'':15} {'':10} ... and {len(files) - 3} more")
parse_cube_files("luts")
    


LUT_3D_SIZE     Count      Files
16              1          Colorify-Cinematic-Standard.cube
32              29         BW3.cube, Presetpro  - Kodacrome 64.cube, Presetpro - Polaroid Color.cube
                           ... and 26 more
33              62         Cinematic-Teal.cube, SoftBlackAndWhite.cube, Parasitic-VLog.cube
                           ... and 59 more
64              3          Presetpro - Rich Tones.cube, Presetpro - Film Fade.cube, Presetpro - Vintage Vibe LUT.cube


In [4]:
#!/usr/bin/env python3
"""
cube_32_to_33.py

Convert .cube 3D LUTs:
- LUT_3D_SIZE 33 -> copy as-is
- LUT_3D_SIZE 32 -> upsample to 33 with trilinear interpolation
"""

from __future__ import annotations

import argparse
import math
import shutil
from dataclasses import dataclass
from pathlib import Path
from typing import List, Optional, Tuple

import numpy as np


@dataclass
class CubeLUT:
    title: Optional[str]
    domain_min: Optional[np.ndarray]  # shape (3,)
    domain_max: Optional[np.ndarray]  # shape (3,)
    size: int
    # data[r, g, b, c] where c in {0,1,2} for RGB, shape (N,N,N,3)
    data: np.ndarray
    preserved_header: List[str]  # any other header lines to keep (non-data, non-title/domain/size)


def _try_parse_rgb_triplet(line: str) -> Optional[Tuple[float, float, float]]:
    parts = line.strip().split()
    if len(parts) < 3:
        return None
    try:
        r, g, b = float(parts[0]), float(parts[1]), float(parts[2])
        return r, g, b
    except ValueError:
        return None


def read_cube(path: Path) -> CubeLUT:
    title = None
    domain_min = None
    domain_max = None
    size = None
    preserved_header: List[str] = []
    triplets: List[Tuple[float, float, float]] = []

    for raw in path.read_text(encoding="utf-8", errors="ignore").splitlines():
        line = raw.strip()
        if not line:
            continue
        if line.startswith("#"):
            # keep comments if you want; for now we ignore them
            continue

        upper = line.upper()

        if upper.startswith("TITLE"):
            # TITLE "something"
            # preserve quotes if present
            title = raw.strip()
            continue

        if upper.startswith("DOMAIN_MIN"):
            parts = line.split()
            if len(parts) >= 4:
                domain_min = np.array([float(parts[1]), float(parts[2]), float(parts[3])], dtype=np.float32)
            continue

        if upper.startswith("DOMAIN_MAX"):
            parts = line.split()
            if len(parts) >= 4:
                domain_max = np.array([float(parts[1]), float(parts[2]), float(parts[3])], dtype=np.float32)
            continue

        if upper.startswith("LUT_3D_SIZE"):
            parts = line.split()
            if len(parts) >= 2:
                size = int(parts[1])
            continue

        if upper.startswith("LUT_1D_SIZE"):
            raise ValueError(f"{path.name}: LUT_1D_SIZE found; this script supports 3D LUTs only.")

        # data line?
        rgb = _try_parse_rgb_triplet(line)
        if rgb is not None:
            triplets.append(rgb)
        else:
            # unknown header-ish line, preserve it
            preserved_header.append(raw.strip())

    if size is None:
        # fallback: infer from triplet count
        n = round(triplets and (len(triplets) ** (1 / 3)) or 0)
        if n <= 0 or n ** 3 != len(triplets):
            raise ValueError(f"{path.name}: Missing LUT_3D_SIZE and cannot infer from {len(triplets)} rows.")
        size = n

    expected = size ** 3
    if len(triplets) < expected:
        raise ValueError(f"{path.name}: Expected {expected} RGB rows, got {len(triplets)}.")
    if len(triplets) > expected:
        # Some files have extra stuff; we only take the first N^3
        triplets = triplets[:expected]

    flat = np.array(triplets, dtype=np.float32)  # shape (N^3, 3)
    data = flat.reshape((size, size, size, 3))   # order: r major, g middle, b fastest (standard .cube)

    return CubeLUT(
        title=title,
        domain_min=domain_min,
        domain_max=domain_max,
        size=size,
        data=data,
        preserved_header=preserved_header,
    )


def trilinear_resample(lut: np.ndarray, out_size: int) -> np.ndarray:
    """
    lut: shape (N, N, N, 3) in r,g,b order
    returns: shape (out_size, out_size, out_size, 3)
    """
    n = lut.shape[0]
    if lut.shape[:3] != (n, n, n) or lut.shape[3] != 3:
        raise ValueError("Expected lut shape (N,N,N,3).")

    out = np.empty((out_size, out_size, out_size, 3), dtype=np.float32)

    # New grid coordinates normalized to [0,1]
    for ir in range(out_size):
        xr = ir / (out_size - 1)
        pr = xr * (n - 1)
        r0 = int(math.floor(pr))
        r1 = min(r0 + 1, n - 1)
        tr = pr - r0

        for ig in range(out_size):
            xg = ig / (out_size - 1)
            pg = xg * (n - 1)
            g0 = int(math.floor(pg))
            g1 = min(g0 + 1, n - 1)
            tg = pg - g0

            for ib in range(out_size):
                xb = ib / (out_size - 1)
                pb = xb * (n - 1)
                b0 = int(math.floor(pb))
                b1 = min(b0 + 1, n - 1)
                tb = pb - b0

                c000 = lut[r0, g0, b0]
                c100 = lut[r1, g0, b0]
                c010 = lut[r0, g1, b0]
                c110 = lut[r1, g1, b0]
                c001 = lut[r0, g0, b1]
                c101 = lut[r1, g0, b1]
                c011 = lut[r0, g1, b1]
                c111 = lut[r1, g1, b1]

                c00 = c000 * (1 - tr) + c100 * tr
                c10 = c010 * (1 - tr) + c110 * tr
                c01 = c001 * (1 - tr) + c101 * tr
                c11 = c011 * (1 - tr) + c111 * tr

                c0 = c00 * (1 - tg) + c10 * tg
                c1 = c01 * (1 - tg) + c11 * tg

                out[ir, ig, ib] = c0 * (1 - tb) + c1 * tb

    return out


def write_cube(path: Path, cube: CubeLUT, size: int, data: np.ndarray) -> None:
    lines: List[str] = []

    # Preserve title if present, else create one
    if cube.title is not None:
        lines.append(cube.title)
    else:
        lines.append(f'TITLE "{path.stem}"')

    # Preserve domains if present
    if cube.domain_min is not None:
        dm = cube.domain_min
        lines.append(f"DOMAIN_MIN {dm[0]:.6f} {dm[1]:.6f} {dm[2]:.6f}")
    if cube.domain_max is not None:
        dx = cube.domain_max
        lines.append(f"DOMAIN_MAX {dx[0]:.6f} {dx[1]:.6f} {dx[2]:.6f}")

    # Preserve any other header lines we saw (optional)
    for h in cube.preserved_header:
        # Avoid duplicating size lines if any slipped in
        if h.upper().startswith("LUT_3D_SIZE"):
            continue
        lines.append(h)

    lines.append(f"LUT_3D_SIZE {size}")
    lines.append("")  # blank line before data

    # Write in standard .cube order: r major, g middle, b fastest
    n = size
    if data.shape != (n, n, n, 3):
        raise ValueError("write_cube: data shape mismatch.")

    for r in range(n):
        for g in range(n):
            for b in range(n):
                rgb = data[r, g, b]
                lines.append(f"{rgb[0]:.6f} {rgb[1]:.6f} {rgb[2]:.6f}")

    path.write_text("\n".join(lines) + "\n", encoding="utf-8")


def process_folder(in_dir: Path, out_dir: Path) -> None:
    out_dir.mkdir(parents=True, exist_ok=True)

    cube_files = sorted(in_dir.glob("*.cube"))
    if not cube_files:
        print(f"No .cube files found in: {in_dir}")
        return

    copied = converted = skipped = 0

    for p in cube_files:
        try:
            cube = read_cube(p)
        except Exception as e:
            print(f"[ERROR] {p.name}: {e}")
            skipped += 1
            continue

        dst = out_dir / p.name

        if cube.size == 33:
            shutil.copy2(p, dst)
            print(f"[COPY] {p.name} (33)")
            copied += 1

        elif cube.size == 32:
            up = trilinear_resample(cube.data, out_size=33)
            write_cube(dst, cube, size=33, data=up)
            print(f"[UPSAMPLE] {p.name} (32 -> 33)")
            converted += 1

        else:
            print(f"[SKIP] {p.name} (LUT_3D_SIZE {cube.size} not in {{32,33}})")
            skipped += 1

    print("\nDone.")
    print(f"  Copied:    {copied}")
    print(f"  Converted: {converted}")
    print(f"  Skipped:   {skipped}")
    print(f"  Output:    {out_dir}")


def run(input_path, output_path) -> None:
    in_dir = Path(input_path).expanduser().resolve()
    out_dir = Path(output_path).expanduser().resolve()

    if not in_dir.exists() or not in_dir.is_dir():
        raise SystemExit(f"Input folder does not exist or is not a directory: {in_dir}")

    process_folder(in_dir, out_dir)



In [6]:
run("./raw_luts", "./ml_luts")

[UPSAMPLE] ARRIVAL.cube (32 -> 33)
[COPY] AUTUMN DULL.cube (33)
[COPY] AUTUMN RICH.cube (33)
[COPY] Arrakis-BMDFilm.cube (33)
[COPY] Arrakis-FLog.cube (33)
[COPY] Arrakis-NLog.cube (33)
[COPY] Arrakis-VLog.cube (33)
[COPY] BRIGHT.cube (33)
[COPY] BW FADED.cube (33)
[COPY] BW HIGH CONTRAST.cube (33)
[UPSAMPLE] BW1.cube (32 -> 33)
[UPSAMPLE] BW10.cube (32 -> 33)
[UPSAMPLE] BW2.cube (32 -> 33)
[UPSAMPLE] BW3.cube (32 -> 33)
[UPSAMPLE] BW4.cube (32 -> 33)
[UPSAMPLE] BW5.cube (32 -> 33)
[UPSAMPLE] BW6.cube (32 -> 33)
[UPSAMPLE] BW7.cube (32 -> 33)
[UPSAMPLE] BW8.cube (32 -> 33)
[UPSAMPLE] BW9.cube (32 -> 33)
[COPY] Bat-BMDFilm.cube (33)
[COPY] Bat-FLog.cube (33)
[COPY] Bat-NLog.cube (33)
[COPY] Bat-VLog.cube (33)
[COPY] BlueArchitecture.cube (33)
[COPY] BlueHour.cube (33)
[COPY] CELLULOID_01_FU_LOW.cube (33)
[COPY] CLASSY.cube (33)
[COPY] Cinematic-Teal.cube (33)
[COPY] Cliff-BMDFilm.cube (33)
[COPY] Cliff-FLog.cube (33)
[COPY] Cliff-NLog.cube (33)
[COPY] Cliff-VLog.cube (33)
[COPY] ColdChr

In [11]:
from pathlib import Path
import re


def _normalize_name(name: str) -> str:
    name = name.lower()
    name = re.sub(r"[^a-z0-9]+", "_", name)
    name = re.sub(r"_+", "_", name)
    return name.strip("_")


def _read_lut_size(text: str) -> int:
    for line in text.splitlines():
        line = line.strip().upper()
        if line.startswith("LUT_3D_SIZE"):
            parts = line.split()
            if len(parts) >= 2:
                return int(parts[1])
    raise ValueError("LUT_3D_SIZE not found")


def normalize_names(working_dir: str) -> None:
    wd = Path(working_dir).expanduser().resolve()
    if not wd.is_dir():
        raise ValueError(f"Not a directory: {wd}")

    for path in wd.glob("*.cube"):
        text = path.read_text(encoding="utf-8", errors="ignore")

        try:
            size = _read_lut_size(text)
        except Exception as e:
            print(f"[SKIP] {path.name}: {e}")
            continue

        base = _normalize_name(path.stem)
        new_name = f"{base}__lut3d_{size}.cube"
        target = path.with_name(new_name)

        if target != path:
            if target.exists():
                target.unlink()
            path.rename(target)

        print(f"[OK] {path.name} → {new_name}")


In [12]:
normalize_names(
    working_dir="./ml_luts",
)


[OK] Licorice-NLog.cube → licorice_nlog__lut3d_33.cube
[OK] BW3.cube → bw3__lut3d_33.cube
[OK] NaturalBoost.cube → naturalboost__lut3d_33.cube
[OK] STARK.cube → stark__lut3d_33.cube
[OK] K_TONE Vintage_KODACHROME.cube → k_tone_vintage_kodachrome__lut3d_33.cube
[OK] Arrakis-FLog.cube → arrakis_flog__lut3d_33.cube
[OK] cinematic_teal.cube → cinematic_teal__lut3d_33.cube
[OK] Presetpro  - Kodacrome 64.cube → presetpro_kodacrome_64__lut3d_33.cube
[OK] HardBoost.cube → hardboost__lut3d_33.cube
[OK] CrispAutumn.cube → crispautumn__lut3d_33.cube
[OK] Presetpro - Polaroid Color.cube → presetpro_polaroid_color__lut3d_33.cube
[OK] parasitic_vlog.cube → parasitic_vlog__lut3d_33.cube
[OK] Licorice-FLog.cube → licorice_flog__lut3d_33.cube
[OK] LushGreen.cube → lushgreen__lut3d_33.cube
[OK] Bat-BMDFilm.cube → bat_bmdfilm__lut3d_33.cube
[OK] BW HIGH CONTRAST.cube → bw_high_contrast__lut3d_33.cube
[OK] WASH.cube → wash__lut3d_33.cube
[OK] BW2.cube → bw2__lut3d_33.cube
[OK] BlueArchitecture.cube → blue